In [1]:
# Importação das bibliotecas
from scipy.stats import loguniform, uniform
import numpy as np
import pandas as pd
import os

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV

from SwarmPackagePy import gwo
import pyswarms as ps

In [2]:
# Tratamento das imagens
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
im_size = 224

street_types = ['clean', 'litter', 'recycle']
path = '../Dataset/'
path_test = '../Dataset_Test/'

In [4]:
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: recycle    1500
clean      1460
litter     1364
dtype: int64


In [5]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.15, random_state=415)

print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (3675, 224, 224, 3)
Validation shape: (649, 224, 224, 3)
Test shape: (1081, 224, 224, 3)


In [6]:
# Definição, compilação e treinamento
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dropout(0.18)(x)
x = Dense(len(street_types), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(0.000495),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

history = model.fit(train_x, 
                    train_y, 
                    epochs=5,
                    validation_data=(val_x, val_y))

Epoch 1/5
115/115 [==============================] - 472s 4s/step - loss: 0.2593 - accuracy: 0.9018 - val_loss: 0.1827 - val_accuracy: 0.9414
Epoch 2/5
115/115 [==============================] - 523s 5s/step - loss: 0.0910 - accuracy: 0.9722 - val_loss: 0.1321 - val_accuracy: 0.9692
Epoch 3/5
115/115 [==============================] - 551s 5s/step - loss: 0.0455 - accuracy: 0.9905 - val_loss: 0.1194 - val_accuracy: 0.9738
Epoch 4/5
115/115 [==============================] - 556s 5s/step - loss: 0.0303 - accuracy: 0.9956 - val_loss: 0.1251 - val_accuracy: 0.9738
Epoch 5/5
115/115 [==============================] - 563s 5s/step - loss: 0.0233 - accuracy: 0.9962 - val_loss: 0.1213 - val_accuracy: 0.9769


In [7]:
# Avaliação e análise
test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded, verbose=1)
print(f"\nAcurácia no teste: {test_accuracy*100:.2f}%\n")

y_pred_test = model.predict(test_images)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

confusion_mtx = confusion_matrix(test_labels_encoded, y_pred_test_classes)
print("Matriz de Confusão:")
print(confusion_mtx)

class_report = classification_report(test_labels_encoded, y_pred_test_classes, target_names=label_encoder.classes_)
print("\nRelatório de Classificação:")
print(class_report)

y_true_binarized = to_categorical(test_labels_encoded)

for i in range(y_true_binarized.shape[1]):
    auc_score = roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i])
    print(f'AUC para a classe {label_encoder.classes_[i]}: {auc_score:.2f}')

34/34 [==============================] - 142s 4s/step - loss: 0.1263 - accuracy: 0.9593

Acurácia no teste: 95.93%

34/34 [==============================] - 139s 4s/step
Matriz de Confusão:
[[331  26   8]
 [  4 336   1]
 [  2   3 370]]

Relatório de Classificação:
              precision    recall  f1-score   support

       clean       0.98      0.91      0.94       365
      litter       0.92      0.99      0.95       341
     recycle       0.98      0.99      0.98       375

    accuracy                           0.96      1081
   macro avg       0.96      0.96      0.96      1081
weighted avg       0.96      0.96      0.96      1081

AUC para a classe clean: 0.99
AUC para a classe litter: 1.00
AUC para a classe recycle: 1.00


In [8]:
# Armazenamento do Excel e Modelo
shapes_data = {
    'Conjunto': ['Treino', 'Validação', 'Teste'],
    'Formato': [train_x.shape, val_x.shape, test_images.shape]
}
df_shapes = pd.DataFrame(shapes_data)

df_confusion_mtx = pd.DataFrame(confusion_mtx, index=label_encoder.classes_, columns=label_encoder.classes_)

report_data = classification_report(test_labels_encoded, y_pred_test_classes, target_names=label_encoder.classes_, output_dict=True)
df_class_report = pd.DataFrame(report_data).transpose()

auc_scores = {label_encoder.classes_[i]: roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i]) for i in range(y_true_binarized.shape[1])}
df_auc_scores = pd.DataFrame(list(auc_scores.items()), columns=['Classe', 'AUC'])

with pd.ExcelWriter('IC_Project_Fase3_Simple_Excel.xlsx') as writer:
    df_shapes.to_excel(writer, sheet_name='Formas dos Conjuntos', index=False)
    df_confusion_mtx.to_excel(writer, sheet_name='Matriz de Confusão')
    df_class_report.to_excel(writer, sheet_name='Relatório de Classificação')
    df_auc_scores.to_excel(writer, sheet_name='AUC por Classe', index=False)

print("Resultados exportados para o arquivo 'IC_Project_Fase3_Simple_Excel.xlsx'")

Resultados exportados para o arquivo 'IC_Project_Fase3_Simple_Excel.xlsx'


In [9]:
model_save_path = 'IC_Project_Fase3_Simple_SaveModel.h5'

model.save(model_save_path)
print(f"Modelo salvo com sucesso em: {model_save_path}")

Modelo salvo com sucesso em: IC_Project_Fase3_Simple_SaveModel.h5
